В первую очередь хочу сказать, что это мой первый опыт обучения любой модели машинного обучения в принципе и первое столкновение с NLP и анализом текста, смотрел несколько видео по табличному машинному обучению

Начнем

Сначала нужно импортировать все необходимые пакеты и модули

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import cross_val_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Мурад\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Мурад\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Далее загружаем данные для обучения модели train и для построения предсказаний test

In [2]:
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')

In [3]:
train.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


In [4]:
test.head()

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,0
1,Ночью под Минском на президентской горе Белара...,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0


Определим переменные

In [5]:
X_train_init = train.title
y_train = train.is_fake
X_test_init = test.title

In [6]:
X_train_init

0       Москвичу Владимиру Клутину пришёл счёт за вмеш...
1       Агент Кокорина назвал езду по встречке житейск...
2       Госдума рассмотрит возможность введения секрет...
3       ФАС заблокировала поставку скоростных трамваев...
4       Против Навального завели дело о недоносительст...
                              ...                        
5753    Эдди Чемберс получил сотрясение мозга в бою с ...
5754    Правительство застроит Россию нефтепродуктопро...
5755    «Мне стыдно перед дедом»: новый канцлер ФРГ об...
5756     Туркмения в декабре начнет поставки газа в Китай
5757    Бывший тренер Локомотива возглавил нальчикский...
Name: title, Length: 5758, dtype: object

Теперь нужно провести предобработку и убрать все множественные пробелы, единичные символы, все специальные символы, также привести все к нижнему регистру, и, наконец, постараться максимально привести к единой форме одни и те же слова, но стоящие в разных формах

In [7]:
documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X_train_init)):
    document = re.sub(r'\W', ' ', str(X_train_init[sen]))
    
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    document = document.lower()
    
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document).strip()
    
    documents.append(document)

In [8]:
documents

['москвичу владимиру клутину пришёл счёт за вмешательство в американские выборы',
 'агент кокорина назвал езду по встречке житейской историей',
 'госдума рассмотрит возможность введения секретных статей уголовного кодекса',
 'фас заблокировала поставку скоростных трамваев для москвы',
 'против навального завели дело о недоносительстве на волкова',
 'российским студентам запретят учиться за рубежом',
 'путин пишет книгу об истории украины',
 'россияне обхитрили рост цен',
 'звезда ворониных раскрыл подробности о своем состоянии',
 'microsoft объявила дату выхода очков дополненной реальности hololens',
 'марат хуснуллин призвал прописать в законе понятие многонационального дома',
 'для проведения профилактических работ с 15 по 19 сентября в россии отключат интернет',
 'в россии введут обязательный техосмотр садового инвентаря',
 'топилин провозгласил окончание зарплатного кризиса в россии',
 'режиссера алексея германа наградили орденом',
 'олаф шольц дед рассказывал какие вкусные на укра

Также проверил варианты без предобработки, варианты с стеммингом через Snowball и Porterstemmer

Далее нужно превратить набор слов и предложений в какую-то "измеряемую" форму. 

Начнем с того, что сделаем просто мешок слов при помощи CountVectorizer

In [9]:
vectorizer_train = CountVectorizer(max_features=90000, min_df=0, max_df=0.55,
ngram_range=(1,6), analyzer='char_wb')
X_train = vectorizer_train.fit_transform(documents)

In [10]:
X_train

<5758x90000 sparse matrix of type '<class 'numpy.int64'>'
	with 1287427 stored elements in Compressed Sparse Row format>

Также я проверил, как влияют стоп-слова русского и английского языка и вместе. Скоры были ниже

Сразу скажу, что фичи подобраны опытным путем вручную.

Я хотел использовать пайплайны и гридсерч, но это занимало страшно много времени. На попытки я потратиk в сумме около 8 часов только обучения помимо того, что писал код. Половину на своем ноуте, а затем уже на Colab

Модели я проверял на просто мешке слов тоже, скоры ниже

Далее переведем мешок слов в вид TF IDF c помощью TfidfTransformer(). Также пробовал TfidfVectorizer(), но он показал результаты хуже

In [11]:
tfidfconverter = TfidfTransformer()
X_train = tfidfconverter.fit_transform(X_train)

In [12]:
X_train

<5758x90000 sparse matrix of type '<class 'numpy.float64'>'
	with 1287427 stored elements in Compressed Sparse Row format>

Все, данные теперь готовы. Могло быть лучше, конечно, и я даже знаю, в чем (пробелы после слов в мешке слов), но время поджимало

Переходим к выбору модели обучения

Скажем так, перед выходом в финал были протестированы и RandomForest, и SGDClassifier, и другие Байесовы классификаторы

По итогу лучше скоры показали BernoulliNB, MultinomialNB и LogisticRegression

Начнем с логистической регрессии. Фичи подобраны опытным путем. Как говорил, хотел через пайплайн и гридсерч, но очень много времени ушло

In [13]:
log_reg = LogisticRegression(penalty='none')

In [14]:
log_reg

LogisticRegression(penalty='none')

Теперь посчитаем скор на кросс-валидации, чтобы избежать погрешности переобучения.

Размеры train сета и test ~ 7:1, поэтому на кросс-валидации возьмем 8 частей

In [15]:
(train.shape[0] + test.shape[0]) / test.shape[0]

6.758

Также важно проставить scoring='f1', так как мы работаем на его увеличение, ну и включить все мощности тоже не помешает

In [16]:
cross_val_score(log_reg, X_train, y_train, cv=8, scoring='f1', n_jobs=-1).mean()

0.8743198330986877

Теперь разберем наивный байес

Для него входные фичи для мешка слов и тф идф немного другие

In [17]:
vectorizer_train = CountVectorizer(max_features=74000, min_df=0, max_df=0.9,
ngram_range=(1,6), analyzer='char_wb')
X_train = vectorizer_train.fit_transform(documents)

In [18]:
tfidfconverter = TfidfTransformer()
X_train = tfidfconverter.fit_transform(X_train)

In [19]:
mnNB = MultinomialNB(alpha=0.09)

In [20]:
cross_val_score(mnNB, X_train, y_train, cv=8, scoring='f1', n_jobs=-1).mean()

0.8610630519978264

Ну и наконец, наивный байес Бернулли

In [21]:
bNB = BernoulliNB(alpha=0.09)

In [22]:
cross_val_score(bNB, X_train, y_train, cv=8, scoring='f1', n_jobs=-1).mean()

0.8836120922161603

Как видим, на кросс-валидации лучший результат показал BernoulliNB - 0.8836, затем LogisticRegression - 0.8743 и затем уже MultinomialNB - 0.8610

Теперь надо подготовить тестовый датасет

In [23]:
documents_test = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X_test_init)):
    document = re.sub(r'\W', ' ', str(X_test_init[sen]))
    
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    document = document.lower()
    
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents_test.append(document)

In [24]:
X_test = vectorizer_train.transform(documents_test)

In [25]:
X_test = tfidfconverter.fit_transform(X_test)

Теперь перейдем к обучнению моделей

In [26]:
log_reg.fit(X_train, y_train)

LogisticRegression(penalty='none')

In [27]:
mnNB.fit(X_train, y_train)

MultinomialNB(alpha=0.09)

In [28]:
bNB.fit(X_train, y_train)

BernoulliNB(alpha=0.09)

сведем все предсказания всех трех моделей в одну таблицу

In [29]:
final = test.merge(pd.DataFrame(pd.Series(bNB.predict(X_test))), left_index=True, right_index=True) \
.rename(columns={'is_fake':'none', 0:'bernoulli'}).loc[:, ['title', 'bernoulli']]

In [30]:
final.head()

,title,bernoulli
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",1
4,Акции Google рухнули после объявления о переза...,1


In [31]:
final = final.merge(pd.DataFrame(pd.Series(log_reg.predict(X_test))), left_index=True, \
right_index=True).rename(columns={0:'log_reg'})

In [32]:
final.head(5)

,title,bernoulli,log_reg
0,Роскомнадзор представил реестр сочетаний цвето...,1,1
1,Ночью под Минском на президентской горе Белара...,1,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1,1
3,"Сельская церковь, собравшая рекордно низкое ко...",1,1
4,Акции Google рухнули после объявления о переза...,1,1


In [33]:
final = final.merge(pd.DataFrame(pd.Series(mnNB.predict(X_test))), left_index=True, \
right_index=True).rename(columns={0:'mnNB'})

In [34]:
final.head(5)

,title,bernoulli,log_reg,mnNB
0,Роскомнадзор представил реестр сочетаний цвето...,1,1,1
1,Ночью под Минском на президентской горе Белара...,1,1,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1,1,1
3,"Сельская церковь, собравшая рекордно низкое ко...",1,1,1
4,Акции Google рухнули после объявления о переза...,1,1,1


После я подумал, как ведут эти три модели с каждой новостью

In [35]:
final['common'] = final.bernoulli + final.log_reg + final.mnNB

In [36]:
final[final.common == 3]

,title,bernoulli,log_reg,mnNB,common
0,Роскомнадзор представил реестр сочетаний цвето...,1,1,1,3
1,Ночью под Минском на президентской горе Белара...,1,1,1,3
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1,1,1,3
3,"Сельская церковь, собравшая рекордно низкое ко...",1,1,1,3
4,Акции Google рухнули после объявления о переза...,1,1,1,3
...,...,...,...,...,...
990,Валентина Терешкова предложила сделать Рассказ...,1,1,1,3
992,Россиян планируют перевести на шестидневную ра...,1,1,1,3
996,В День Победы стратегические ракетоносцы Ту-16...,1,1,1,3
997,СК возбудил дело против авиакомпании «Победа» ...,1,1,1,3


462 новости все три модели определили как фейк

In [37]:
final[final.common == 0]

,title,bernoulli,log_reg,mnNB,common
5,Курс доллара вырос до исторического максимума,0,0,0,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0,0,0,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0,0,0,0
9,Учреждена театральная премия имени Гарольда Пи...,0,0,0,0
10,Песков опроверг информацию о слиянии Минфина и...,0,0,0,0
...,...,...,...,...,...
987,Сафин выходит в третий круг после травмы сопер...,0,0,0,0
988,Центробанк отозвал лицензии у двух московских ...,0,0,0,0
989,Россия и Венесуэла договорились о сотрудничест...,0,0,0,0
991,Бывшему гендиректору Google подняли зарплату в...,0,0,0,0


397 новостей все три модели определили как не фэйк

То есть я уже с какой степенью уверенности могу сказать, что 859 новостей, скорее всего, определены правильно. То есть, на accuracy 85.9 могу рассчитывать

In [38]:
final[final.common == 2].query('bernoulli == 1 & log_reg == 1')

,title,bernoulli,log_reg,mnNB,common
62,«Лучший коньяк в мире - дагестанский»: в Париж...,1,1,0,2
114,Ультраправые консерваторы потребовали уменьшит...,1,1,0,2
211,«Газпром» объявил тарифы на бесплатную газифик...,1,1,0,2
348,Северокорейская вона стала первой в новейшей и...,1,1,0,2
450,Россия из-за критики передумала отправлять Man...,1,1,0,2
493,Минпромторг разрешил российским производителям...,1,1,0,2
553,«Лягушатникам лучше не нарываться»: Мария Заха...,1,1,0,2
620,Китайский космический зонд «Тянвэнь-1» сфотогр...,1,1,0,2
704,«Газпром» бесплатно газифицирует дома выходцев...,1,1,0,2
746,Средняя зарплата китайских заключённых превыси...,1,1,0,2


12 новостей две более сильные модели определили как фэйк

In [39]:
final[final.common == 1].query('bernoulli == 0 & log_reg == 0')

,title,bernoulli,log_reg,mnNB,common
53,«Росатом» построит АЭС в Афганистане,0,0,1,1
58,За неделю московские квартиры подорожали на пр...,0,0,1,1
69,В Петербурге открылся музей сосулек,0,0,1,1
218,Главу итальянского партнера «Сухого» арестовал...,0,0,1,1
255,В Москве похоронили Людмилу Гурченко,0,0,1,1
298,В длинный список Бунинской премии вошли 60 книг,0,0,1,1
301,Для концерта «Океана Ельзи» в Петербурге нашла...,0,0,1,1
317,Электронный Воннегут опередит бумажного,0,0,1,1
396,#РИФМЫ НЕДЕЛИ,0,0,1,1
466,Госдума ратифицировала договор о техпомощи ООН,0,0,1,1


29 новостей две более сильные модели определили как не фэйк

То есть, с очень большой долей вероятности 900 новостей определены верно двумя более сильными моделями

Учитывая, что Бернулли сама по себе и подтвержденная другими моделями имеет высокий скор(я понимаю, что это accuracy), я выберу ее. Риски меньше. Но. Я мог бы продолжить логику того, чтобы с каким-то допущением совместить предикты всех трех моделей.

Но тут в дело вступила новость под номером 17

In [40]:
final.iloc[[17]]

,title,bernoulli,log_reg,mnNB,common
17,Биткоин затмил Ким Кардашьян,0,1,1,2


А судя по всему это не фейк, и только бернулли это показал. Конечно, это может быть случайность, но пока все говорит за бернулли)))

Поэтому:

In [41]:
predictions = final.loc[:, ['title', 'bernoulli']].rename(columns={'bernoulli': 'is_fake'})

Посмотрим на распределение ответов

In [42]:
predictions.groupby('is_fake').count()

,title
is_fake,
0,496
1,504


In [43]:
predictions.to_csv('predictions.tsv', sep='\t', index=False)